## Pre-Processing Data Tweets  Training

1. Hapus Mention (@username)
2. Hapus Hastag (#)
3. Hapus URL
4. Hapus Emoticon
5. Cek Duplicate

In [ ]:
pip install nest_asyncio

In [ ]:
pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.3 MB/s eta 0:00:00


In [ ]:
pip install scikeras

In [ ]:
from keras.utils import pad_sequences

In [ ]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
#import some libraries

import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
seed =0
np.random.seed(seed)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = "whitegrid")
plt.style.use('fivethirtyeight')
import nest_asyncio
nest_asyncio.apply()
import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import nltk
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, LSTM
from keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load data

In [ ]:
#Load data
tweets = pd.read_excel('/content/drive/MyDrive/Trainning Tweets (Raw).xlsx')

In [ ]:
tweets.head()

,created_at,id_str,full_text,quote_count,reply_count,retweet_count,favorite_count,lang,user_id_str,conversation_id_str,username,tweet_url
0,Wed Nov 08 07:12:11 +0000 2023,1720000000000000000,"Ganjar Pranowo Hormati Putusan MKMK, Enggan Ko...",0,0,0,1,in,47596019,1720000000000000000,liputan6dotcom,https://twitter.com/liputan6dotcom/status/1722...
1,Tue Nov 07 00:00:00 +0000 2023,1720000000000000000,HEADLINE: Penetapan Pasangan Capres-Cawapres d...,0,0,0,1,in,47596019,1720000000000000000,liputan6dotcom,https://twitter.com/liputan6dotcom/status/1721...
2,Mon Nov 06 17:01:34 +0000 2023,1720000000000000000,HEADLINE: Penetapan Pasangan Capres-Cawapres d...,0,0,0,0,in,47596019,1720000000000000000,liputan6dotcom,https://twitter.com/liputan6dotcom/status/1721...
3,Mon Nov 06 08:11:05 +0000 2023,1720000000000000000,Cek Fakta: Tidak Benar Ridwan Kamil Kerahkan S...,0,2,0,4,in,47596019,1720000000000000000,liputan6dotcom,https://twitter.com/liputan6dotcom/status/1721...
4,Mon Nov 06 06:47:05 +0000 2023,1720000000000000000,VIDEO: Cak Imin Incar Nomor Urut Satu di Pilpr...,0,6,10,67,in,47596019,1720000000000000000,liputan6dotcom,https://twitter.com/liputan6dotcom/status/1721...


### Cleaning

In [ ]:
# Kamus singkatan bahasa Indonesia
abbreviation_mapping = {
    "bhw": "bahwa",
    "kpd": "kepada",
    "bsk": "besok",
    "sbg": "sebagai",
    "spt": "seperti",
    "jd": "jadi",
    "tjd": "terjadi",
    "tp": "tapi",
    "dl": "dulu",
    "krn": "karena",
    "blm": "belum",
    "sm": "sama",
    "kmrn": "kemarin",
    "pdhl": "padahal",
    "gmn": "bagaimana",
    "mhn": "mohon",
    "trs": "terus",
    "bnyk": "banyak",
    "jln": "jalan",
    "tpi": "tapi",
    "nnti": "nanti",
    "mlm": "malam",
    "sy": "saya",
    "hnya": "hanya",
    "dg": "dengan",
    "yg": "yang",
    "bkn": "bukan",
    "km": "kamu",
    "gw": "gue",
    "hrs": "harus",
    "gk": "gak",
    "sdh": "sudah",
    "aja": "saja",
    "dlm": "dalam",
    "dgn": "dengan",
    "knp": "kenapa",
    "dr": "dari",
    "klo": "kalau",
    "lgsg": "langsung",
    "cm": "cuma",
    "ntn": "nonton",
    "smpt": "sempat",
    "kl": "kalau",
    "bbrp": "beberapa",
    "mnt": "minta",
    "plg": "paling",
    "tmbh": "tambahan",
    "bwt": "buat",
    "trnyata": "ternyata",
    "pny": "punya",
    "gpp": "gaperlu",
    "bls": "balas",
    "tpk": "terpaksa",
    "ntar": "nanti",
    "bngt": "banget",
    "wkt": "waktu",
    "skrg": "sekarang",
    "btw": "by the way",
    "ngrasain": "merasakan",
    "lgs": "langsung",
    "nny": "annya",
    "mgkn": "mungkin",
    "drpd": "daripada",
    "ksh": "kasih",
    "bknnya": "bukannya",
    "mnrt": "menurut",
    "nyiapin": "menyiapkan",
    "nulis": "menulis",
    "lbh": "lebih",
    "tmbah": "tambah",
    "lg": "lagi",
    "skt": "sakit",
    "blkg": "belakang",
    "blg": "bilang",
    "tmn": "teman",
    "krj": "kerja",
    "syg": "sayang",
    "mggu": "minggu",
    "lmyn": "lumayan",
    "brp": "berapa",
    "msh": "masih",
    "tdk": "tidak",
    "lw": "lu",
    "pdhl": "padahal",
    "skl": "sekolah",
    "nih": "ini",
    "kdg": "kadang",
    "bgt": "banget",
    "sj": "saja",
    "gt": "gitu",
    "lgsg": "langsung",
    "kgk": "enggak",
    "ssuai": "sesuai",
    "trpksa": "terpaksa",
    "mnt": "minta",
    "sll": "selalu",
    "pdt": "pede",
    "ny": "nya",
    "klu": "kalau",
    "p" : "pak",
    "org" : "orang",
    "pst" : "pasti"
}

# # Contoh penggunaan
# text_with_abbreviations = "Saya mau beli buku dg harga murah."
# text_without_abbreviations = replace_abbreviations(text_with_abbreviations)
# print(text_without_abbreviations)

In [ ]:
## Some functions for preprocessing text

def cleaningText(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text) #remove mentions
  text = re.sub(r'#[A-Za-z0-9]+', '', text) #remove hastag
  text = re.sub(r'RT[\s]', '', text) # remove RT
  text = re.sub(r"http\S+", '', text) # remove link
  text = re.sub(r'[0-9]+', '', text) # hapus angka
  text = re.sub(r'(.)\1{2,}', r'\1', text)  #hapus huruf berlebih
  text = re.sub(r'\b(ha)+\b', '', text) # hapus ekspresi berlebih
  text = re.sub(r'\b(he)+\b', '', text)
  text = re.sub(r'\b(hi)+\b', '', text)
  text = re.sub(r'\b(ho)+\b', '', text)
  text = re.sub(r'\b(hu)+\b', '', text)
  text = re.sub(r'\b(kw)+\b', '', text)
  text = re.sub(r'\b(wk)+\b', '', text)
  text = re.sub(r'\b(xi)+\b', '', text)
  text = re.sub(r'HEADLINE: ', '', text)
  text = re.sub(r'Cek Fakta: ', '', text)
  text = re.sub(r'VIDEO: ', '', text)
  text = re.sub(r'NEWS Terpopuler: ', '', text)
  text = re.sub(r'&gt', '', text)

  emoj = re.compile("["
  u"\U0001F600-\U0001F64F" # emoticon
  u"\U0001F300-\U0001F5FF" # simbol & piktograf
  u"\U0001F680-\U0001F6FF" # transport & simbol peta
  u"\U0001F1E0-\U0001F1FF" # flags (iOS)
  u"\U00002500-\U00002BEF" # Karakter china
  u"\U00002702-\U000027B0"
  u"\U00002702-\U000027B0"
  u"\U000024C2-\U0001F251"
  u"\U0001f926-\U0001f937"
  u"\U00010000-\U0010ffff"
  u"\u2640-\u2642"
  u"\u2600-\u2B55"
  u"\u200d"
  u"\u23cf"
  u"\u23e9"
  u"\u231a"
  u"\ufe0f" # dingbats
  u"\u3030"
  "]+", re.UNICODE)
  text = re.sub(emoj, '', text)

  text = text.replace('\n', ' ') #replace new line into space
  text = text.translate(str.maketrans('', '', string.punctuation)) #remove all punctuations
  text = text.strip(' ') #remove characters space from both left #right text
  return text

def casefoldingText(text): #Converting all the characters in a text into lower case
    text = text.lower()
    return text

def replace_abbreviations(text):
    words = text.split()
    replaced_words = [abbreviation_mapping[word] if word in abbreviation_mapping else word for word in words]
    return ' '.join(replaced_words)

In [ ]:
pip install NLTK Downloader

  Preparing metadata (setup.py) ... done
  Created wheel for Downloader: filename=downloader-0.98-py3-none-any.whl size=8132 sha256=da25a01eb467bb3de14c19629dde220338e0e4b0d2a8b375f7b0542a560fef7f
  Stored in directory: /root/.cache/pip/wheels/fe/f0/7b/f27dc70e81690fbda0203a40046f3b72d5096be107a749adb3
Successfully built Downloader


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.3 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16351 sha256=802ba64cc3de1ba52d75780f192215812092467bb820eab808a66cd7a554ca18
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling ch

In [ ]:
import googletrans
print(googletrans.__version__)

3.1.0-alpha


In [ ]:
from googletrans.client import Translator
translator=Translator()

In [ ]:
#Preprocessing tweets data
tweets['text_clean'] = tweets['full_text'].apply(cleaningText)
tweets['text_clean'] = tweets['text_clean'].apply(casefoldingText)
tweets['text_clean'] = tweets['text_clean'].apply(replace_abbreviations)
# tweets.drop(['full_text'], axis = 1, inplace = True)
tweets['text_eng']=tweets['text_clean'].str.encode('ascii', 'ignore').apply(translator.translate, src='id', dest='en') # translate from indo to english
tweets['text_eng']=tweets['text_eng'].apply(getattr, args=('text',))

In [ ]:
for text in tweets['text_eng']:
    print(text)

b'ganjar Pranowo respects mkmk's decision reluctant to comment on gibran continuing to run for presidential election'
b'determination of presidential and vice-presidential candidates in the presidential election to design the presidential election team'
b'determination of presidential and vice-presidential candidates in the presidential election to design the presidential election team'
b'it's not true that Ridwan Kamil mobilized all West Java residents to win Anies Baswedan in the presidential election'
b'cak Imin aims for number one in the presidential election'
b'waiting for the announcement of the presidential and vice-presidential candidates for the presidential election competing for popular figures'
b'accepts Prabowogibran's registration in the presidential election, KPU is sued for IDR trillion'
b'accepts Prabowogibran's registration in the presidential election, KPU is sued for IDR trillion'
b'exciting: KPU sued for IDR trillion after accepting Prabowogibran's presidential ele

In [ ]:
tweets['text_eng'] = tweets['text_eng'].apply(lambda x: x[2:] if x.startswith('b') else x) # remove 'b' in initial text

In [ ]:
print('data duplikat setelah dicleaning : ', tweets['text_clean'].duplicated().sum())
print('data duplikat setelah dicleaning dan ditranslate : ', tweets['text_eng'].duplicated().sum())

data duplikat setelah dicleaning :  0
data duplikat setelah dicleaning dan ditranslate :  0


In [ ]:
tweets['text_eng'].isnull().sum()

0

In [ ]:
#drop duplicates tweets
tweets.drop_duplicates(subset = 'text_clean', keep='first', inplace = True)
tweets.drop_duplicates(subset = 'text_eng', keep='first', inplace = True)
print('data duplikat cleaning setelah dihapus : ', tweets['text_clean'].duplicated().sum())
print('data duplikat eng setelah dihapus : ', tweets['text_eng'].duplicated().sum())

data duplikat cleaning setelah dihapus :  0
data duplikat eng setelah dihapus :  0


In [ ]:
tweets.shape

(2270, 14)

In [ ]:
tweets.reset_index(drop=True, inplace=True)

In [ ]:
# #Tokenizing, Filtering(Stopwords), Stemming (Subwords Tokenizing)
# tweets['text_preprocessed'] = tweets['text_clean'].apply(tokenizingText)
# tweets['text_preprocessed'] = tweets['text_preprocessed'].apply(filteringText)
# tweets['text_preprocessed'] = tweets['text_preprocessed'].apply(stemmingText)

In [ ]:
# tweets['text_preprocessed'].duplicated().sum()

In [ ]:
# # cek data yang duplikat
# tweets['is_duplicated']=tweets.duplicated(subset=['text_preprocessed'], keep=False)
# duplicates_tweets=tweets[tweets['is_duplicated']]
# duplicates_tweets

In [ ]:
# #drop duplicates/spams tweets
# tweets.drop_duplicates(subset = 'text_preprocessed', keep='first', inplace = True)

In [ ]:
# tweets.text_preprocessed.duplicated().sum()

In [ ]:
# tweets.shape

In [ ]:
# tweets.to_excel('Tweets Training Cleaning.xlsx', index=False)

### Labeling

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")
sentiments = SentimentIntensityAnalyzer()
tweets['Positive'] = [sentiments.polarity_scores(i)["pos"] for i in tweets["text_eng"]]
tweets['Negative'] = [sentiments.polarity_scores(i)["neg"] for i in tweets["text_eng"]]
tweets['Neutral'] = [sentiments.polarity_scores(i)["neu"] for i in tweets["text_eng"]]
tweets['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in tweets["text_eng"]]

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
score = tweets ["Compound"].values
sentiment = []
for i in score:
  if i >= 0.41 :
      sentiment.append('positif')
  elif i <= -0.25 :
        sentiment.append('negatif')
  else:
      sentiment.append('netral')
tweets["Sentiment"] =sentiment

In [ ]:
df1=tweets[['text_clean', 'Sentiment', 'Compound']]
df1[(df1['Compound']<=-0.2)]

,text_clean,Sentiment,Compound
25,mk tolak gugatan batas usia capres maksimal ta...,negatif,-0.6249
48,jokowi enggan berkomentar soal putusan mk terk...,negatif,-0.4404
57,megawati buka suara soal wacana duet prabowoga...,negatif,-0.4588
78,mengapa belum ada restu dari pks terkait ditet...,negatif,-0.3724
81,sby mengaku telah diingatkan oleh sahabatsahab...,netral,-0.2023
...,...,...,...
2416,benarkah aksi yang dilakukan ganjar pranowo de...,negatif,-0.4019
2417,langkah presiden joko widodo yang mengorganisa...,negatif,-0.8126
2426,presiden menyatakan adanya potensi riakriak ga...,negatif,-0.7269
2433,posisi yang menjabat sebagai gubernur jawa ten...,negatif,-0.8020


In [ ]:
tweets.Sentiment.value_counts()

netral     1220
positif     851
negatif     199
Name: Sentiment, dtype: int64

In [ ]:
# tweets.to_excel('Tweets Training Label.xlsx', index=False)

In [ ]:
# # Load lexicon positive and negative data

# import csv

# # Load positive lexicon data from tsv file
# lexicon_positive = dict()
# with open('/content/drive/MyDrive/positive.tsv', 'r', encoding='utf-8', newline='') as tsvfile:
#     reader = csv.reader(tsvfile, delimiter='\t')

#     # Skip header
#     next(reader, None)

#     for row in reader:
#         lexicon_positive[row[0]] = int(row[1])

# # Load negative lexicon data from tsv file
# lexicon_negative = dict()
# with open('/content/drive/MyDrive/negative.tsv', 'r', encoding='utf-8', newline='') as tsvfile:
#     reader = csv.reader(tsvfile, delimiter='\t')

#     # Skip header
#     next(reader, None)

#     for row in reader:
#         lexicon_negative[row[0]] = int(row[1])

# # Function to determine sentiment polarity of tweets
# def sentiment_analysis_lexicon_indonesia(text):
#     score = 5
#     for word in text:
#         if word in lexicon_positive:
#             score += lexicon_positive[word]
#     for word in text:
#         if word in lexicon_negative:
#             score += lexicon_negative[word]

#     polarity = 'positive' if score > 0 else ('negative' if score < 0 else 'neutral')
#     return score, polarity

In [ ]:
# # Load lexicon positive and negative data
# lexicon_positive=dict()
# import csv
# with open('/content/drive/MyDrive/positive.csv', 'r') as csvfile:
#   reader=csv.reader(csvfile)
#   for row in reader:
#     lexicon_positive[row[0]]=int(row[1])

# # lexicon_positive = {}

# # with open('/content/drive/MyDrive/positive.csv', 'r') as csvfile:
# #     reader = csv.reader(csvfile, delimiter=';')
# #     header = next(reader)  # Membaca header file CSV
# #     for row in reader:
# #         if len(row) >= 2 and row[1].isdigit():  # Pastikan setiap baris memiliki setidaknya 2 elemen dan elemen kedua adalah bilangan
# #             lexicon_positive[row[0]] = int(row[1])

# lexicon_negative=dict()
# import csv
# with open('/content/drive/MyDrive/negative.csv', 'r') as csvfile:
#   reader=csv.reader(csvfile, delimiter=';')
#   for row in reader:
#     lexicon_negative[row[0]]=int(row[1])

# # Function to determine sentiment polarity of tweets
# def sentiment_analysis_lexicon_indonesia(text):
#   #for word in text:
#   score=0
#   for word in text:
#     if (word in lexicon_positive):
#       score=score+lexicon_positive[word]
#   for word in text:
#     if (word in lexicon_negative):
#       score=score+lexicon_negative[word]
#   polarity=''
#   if (score>0):
#     polarity='positive'
#   elif (score<0):
#     polarity='negative'
#   else:
#     polarity='neutral'
#   return score, polarity


In [ ]:
# # Results from determine sentiment polarity of tweets

# results=tweets['text_preprocessed'].apply(sentiment_analysis_lexicon_indonesia)
# results=list(zip(*results))
# tweets['polarity_score']=results[0]
# tweets['polarity']=results[1]
# print(tweets['polarity'].value_counts())

### Tokenizing

In [ ]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2270 entries, 0 to 2269
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   created_at           2270 non-null   object 
 1   id_str               2270 non-null   int64  
 2   full_text            2270 non-null   object 
 3   quote_count          2270 non-null   int64  
 4   reply_count          2270 non-null   int64  
 5   retweet_count        2270 non-null   int64  
 6   favorite_count       2270 non-null   int64  
 7   lang                 2270 non-null   object 
 8   user_id_str          2270 non-null   int64  
 9   conversation_id_str  2270 non-null   int64  
 10  username             2270 non-null   object 
 11  tweet_url            2270 non-null   object 
 12  text_clean           2270 non-null   object 
 13  text_eng             2270 non-null   object 
 14  Positive             2270 non-null   float64
 15  Negative             2270 non-null   f

In [ ]:
def tokenizingText(text): #Tokenizing or splitting a string, text into a list of tokens
    text = word_tokenize(text)
    return text

def filteringText(text): #Remove stopwords in a text
    listStopwords = set(stopwords.words('indonesian'))
    filtered = []
    for txt in text:
      if txt not in listStopwords:
          filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): #Reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = [stemmer.stem(word) for word in text]
    return text

def toSentence(list_words): #Convert list of words into sentence
    sentence = ' '.join(word for word in list_words)
    return sentence

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#Tokenizing, Filtering(Stopwords), Stemming (Subwords Tokenizing)
tweets['text_preprocessed'] = tweets['text_clean'].apply(tokenizingText)
tweets['text_preprocessed'] = tweets['text_preprocessed'].apply(filteringText)
tweets['text_preprocessed'] = tweets['text_preprocessed'].apply(stemmingText)

print('data duplikat setelah ditokenizing : ', tweets['text_preprocessed'].duplicated().sum())

data duplikat setelah ditokenizing :  9


In [ ]:
tweets.drop_duplicates(subset = 'text_preprocessed', keep='first', inplace = True)
tweets['text_preprocessed'].duplicated().sum()

0

In [ ]:
tweets.shape

(2261, 20)

In [ ]:
# tweets.to_excel('Tweets Training Model.xlsx', index=False)

## Pre-Processing Data Tweets Testing

### Load data

In [ ]:
tweets_test=pd.read_excel('/content/drive/MyDrive/Testing Tweets (Raw).xlsx')
tweets_test.head()

,created_at,id_str,full_text,quote_count,reply_count,retweet_count,favorite_count,lang,user_id_str,conversation_id_str,username,tweet_url
0,Sun Nov 12 14:25:00 +0000 2023,1723708475874619904,"Anies Yakin Menang Satu Putaran, Sandiaga Pred...",4,5,2,16,in,69183155,1723708475874619904,detikcom,https://twitter.com/detikcom/status/1723708475...
1,Sun Nov 12 12:55:00 +0000 2023,1723685826473989888,Gibran Optimistis Menang Satu Putaran di Pilpr...,15,42,9,45,in,69183155,1723685826473989888,detikcom,https://twitter.com/detikcom/status/1723685826...
2,Sun Nov 12 09:50:00 +0000 2023,1723639269699249920,Anies Yakin Menang Pilpres Satu Putaran Anies...,58,200,321,1750,in,69183155,1723639269699249920,detikcom,https://twitter.com/detikcom/status/1723639269...
3,Sun Nov 12 08:35:00 +0000 2023,1723620395352280064,Kala PDIP Nilai Bobby Tak Ada Etika karena Dek...,0,10,1,19,in,69183155,1723620395352280064,detikcom,https://twitter.com/detikcom/status/1723620395...
4,Sun Nov 12 07:45:00 +0000 2023,1723607812406640128,"Baliho Ganjar-Mahfud Dicopot, Sandiaga Bicara ...",0,1,0,5,in,69183155,1723607812406640128,detikcom,https://twitter.com/detikcom/status/1723607812...


### Cleaning

In [ ]:
#Preprocessing tweets data
tweets_test['text_clean'] = tweets_test['full_text'].apply(cleaningText)
tweets_test['text_clean'] = tweets_test['text_clean'].apply(casefoldingText)
tweets_test['text_clean'] = tweets_test['text_clean'].apply(replace_abbreviations)
# tweets.drop(['full_text'], axis = 1, inplace = True)
tweets_test['text_eng']=tweets_test['text_clean'].str.encode('ascii', 'ignore').apply(translator.translate, src='id', dest='en') # translate from indo to english
tweets_test['text_eng']=tweets_test['text_eng'].apply(getattr, args=('text',))

In [ ]:
tweets_test['text_eng'] = tweets_test['text_eng'].apply(lambda x: x[2:] if x.startswith('b') else x) # remove 'b' in initial text

In [ ]:
print('data duplikat setelah dicleaning : ', tweets_test['text_clean'].duplicated().sum())
print('data duplikat setelah dicleaning dan ditranslate : ', tweets_test['text_eng'].duplicated().sum())

data duplikat setelah dicleaning :  3
data duplikat setelah dicleaning dan ditranslate :  3


In [ ]:
#drop duplicates tweets
tweets_test.drop_duplicates(subset = 'text_clean', keep='first', inplace = True)
tweets_test.drop_duplicates(subset = 'text_eng', keep='first', inplace = True)
print('data duplikat cleaning setelah dihapus : ', tweets_test['text_clean'].duplicated().sum())
print('data duplikat eng setelah dihapus : ', tweets_test['text_eng'].duplicated().sum())

data duplikat cleaning setelah dihapus :  0
data duplikat eng setelah dihapus :  0


In [ ]:
tweets_test.shape

(611, 14)

In [ ]:
tweets_test.reset_index(drop=True, inplace=True)

### Labeling

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")
sentiments = SentimentIntensityAnalyzer()
tweets_test['Positive'] = [sentiments.polarity_scores(i)["pos"] for i in tweets_test["text_eng"]]
tweets_test['Negative'] = [sentiments.polarity_scores(i)["neg"] for i in tweets_test["text_eng"]]
tweets_test['Neutral'] = [sentiments.polarity_scores(i)["neu"] for i in tweets_test["text_eng"]]
tweets_test['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in tweets_test["text_eng"]]

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
score = tweets_test ["Compound"].values
sentiment = []
for i in score:
  if i >= 0.557 :
      sentiment.append('positif')
  elif i <= -0.02 :
        sentiment.append('negatif')
  else:
      sentiment.append('netral')
tweets_test["Sentiment"] =sentiment

In [ ]:
df2=tweets_test[['text_clean', 'Sentiment', 'Compound']]
# df2[(df2['Sentiment']=='positif')]
df2[(df2['Compound']>0.55) & (df2['Compound']<=0.59)]

,text_clean,Sentiment,Compound
3,kala pdip nilai bobby tak ada etika karena dek...,positif,0.5859
29,ganjar optimistis bali subsisidi suara untuk d...,positif,0.5574
69,prabowo di panggung deklarasi psi ada musik ng...,positif,0.5859
139,canda puan soal nama singkatan ganjarmahfud sa...,positif,0.5574
149,daniel noboa bakal jadi presiden termuda dalam...,positif,0.5719
153,pengusaha pisang jadi presiden ekuador di usia...,positif,0.5719
170,relawan projo resmi mendeklarasikan dukunganny...,positif,0.5574
187,pdip menargetkan persen suara di bangka belitu...,positif,0.5859
221,ketua dpd gerindra jawa timur anwar sadad meng...,positif,0.5719
224,gnpf ulama pa hingga fpi tak lagi memberikan d...,positif,0.5719


In [ ]:
tweets_test.Sentiment.value_counts()

netral     317
positif    242
negatif     52
Name: Sentiment, dtype: int64

In [ ]:
# tweets_test.to_excel('Tweets Testing Label.xlsx', index=False)

### Tokenizing

In [ ]:
tweets_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   created_at           611 non-null    object 
 1   id_str               611 non-null    int64  
 2   full_text            611 non-null    object 
 3   quote_count          611 non-null    int64  
 4   reply_count          611 non-null    int64  
 5   retweet_count        611 non-null    int64  
 6   favorite_count       611 non-null    int64  
 7   lang                 611 non-null    object 
 8   user_id_str          611 non-null    int64  
 9   conversation_id_str  611 non-null    int64  
 10  username             611 non-null    object 
 11  tweet_url            611 non-null    object 
 12  text_clean           611 non-null    object 
 13  text_eng             611 non-null    object 
 14  Positive             611 non-null    float64
 15  Negative             611 non-null    flo

In [ ]:
#Tokenizing, Filtering(Stopwords), Stemming (Subwords Tokenizing)
tweets_test['text_preprocessed'] = tweets_test['text_clean'].apply(tokenizingText)
tweets_test['text_preprocessed'] = tweets_test['text_preprocessed'].apply(filteringText)
tweets_test['text_preprocessed'] = tweets_test['text_preprocessed'].apply(stemmingText)

print('data duplikat setelah ditokenizing : ', tweets_test['text_preprocessed'].duplicated().sum())

data duplikat setelah ditokenizing :  1


In [ ]:
tweets_test.drop_duplicates(subset = 'text_preprocessed', keep='first', inplace = True)
tweets_test['text_preprocessed'].duplicated().sum()

0

In [ ]:
tweets_test.shape

(610, 20)

In [ ]:
# tweets_test.to_excel('Tweets Testing Model fix.xlsx', index=False)